In [ ]:
!cp -r /usr/local/cuda-10.1/include/* /usr/include/

In [ ]:
!pip install git+https://github.com/tommy19970714/Pytorch-Correlation-extension.git@support-operator

  Cloning https://github.com/tommy19970714/Pytorch-Correlation-extension.git (to revision support-operator) to /tmp/pip-req-build-c3n1ccel
  Running command git clone -q https://github.com/tommy19970714/Pytorch-Correlation-extension.git /tmp/pip-req-build-c3n1ccel
  Running command git checkout -b support-operator --track origin/support-operator
  Switched to a new branch 'support-operator'
  Branch 'support-operator' set up to track remote branch 'support-operator' from 'origin'.
  Created wheel for spatial-correlation-sampler: filename=spatial_correlation_sampler-0.3.0-cp37-cp37m-linux_x86_64.whl size=3173003 sha256=9a9f71907662f123152dbbf8590fba34768980c0c5ead6509b40bfe11b9f4413
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gzyjinr/wheels/84/c1/c0/397e3ede95a8df8bf8ff91328fa257d0013eb28d69337c0441
Successfully built spatial-correlation-sampler


# Operator version

In [ ]:
import torch
torch.ops.load_library("/usr/local/lib/python3.7/dist-packages/spatial_correlation_sampler_backend.cpython-37m-x86_64-linux-gnu.so")

In [ ]:
print(torch.ops.my_ops.forward)
print(torch.ops.my_ops.backward)

<built-in method forward of PyCapsule object at 0x7f08c5ce8db0>
<built-in method backward of PyCapsule object at 0x7f08c5990780>


In [ ]:
from torch.autograd import Function
import torch.nn as nn
class SpatialCorrelationSamplerFunction():

    @torch.jit.script
    def forward(input1, input2):

        kH, kW, patchH, patchW, padH, padW, dilationH, dilationW, dilation_patchH, dilation_patchW, dH, dW = 1, 1, 7, 7, 0, 0, 1, 1, 2, 2, 1, 1

        output = torch.ops.my_ops.forward(input1, input2,
                                     kH, kW, patchH, patchW,
                                     padH, padW, dilationH, dilationW,
                                     dilation_patchH, dilation_patchW,
                                     dH, dW)
        return output

In [ ]:
def spatial_correlation_sample(input1,
                               input2,
                               kernel_size=1,
                               patch_size=1,
                               stride=1,
                               padding=0,
                               dilation=1,
                               dilation_patch=1):
    return SpatialCorrelationSamplerFunction.apply(input1, input2,
                                                   kernel_size, patch_size,
                                                   stride, padding, dilation, dilation_patch)

In [ ]:
import torch
import torch.nn as nn

@torch.jit.script
def compute(self, input1, input2):
    return torch.ops.my_ops.forward(input1, input2, 3, 3, 1, 1, 0, 0, 2, 2, 1, 1, 2, 2)

In [ ]:
compute.save("example.pt")

# TEST

In [ ]:
import spatial_correlation_sampler_backend as correlation

def spatial_correlation_sample(input1,
                               input2,
                               kernel_size=1,
                               patch_size=1,
                               stride=1,
                               padding=0,
                               dilation=1,
                               dilation_patch=1):
    return SpatialCorrelationSamplerFunction.apply(input1, input2,
                                                   kernel_size, patch_size,
                                                   stride, padding, dilation, dilation_patch)

class SpatialCorrelationSamplerFunction(Function):

    @staticmethod
    def forward(ctx,
                input1,
                input2,
                kernel_size=1,
                patch_size=1,
                stride=1,
                padding=0,
                dilation=1,
                dilation_patch=1):

        ctx.save_for_backward(input1, input2)
        kH, kW = ctx.kernel_size = _pair(kernel_size)
        patchH, patchW = ctx.patch_size = _pair(patch_size)
        padH, padW = ctx.padding = _pair(padding)
        dilationH, dilationW = ctx.dilation = _pair(dilation)
        dilation_patchH, dilation_patchW = ctx.dilation_patch = _pair(dilation_patch)
        dH, dW = ctx.stride = _pair(stride)

        print(kH, kW, patchH, patchW, padH, padW, dilationH, dilationW, dilation_patchH, dilation_patchW, dH, dW)

        output = correlation.forward(input1, input2,
                                     kH, kW, patchH, patchW,
                                     padH, padW, dilationH, dilationW,
                                     dilation_patchH, dilation_patchW,
                                     dH, dW)

        return output

    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        input1, input2 = ctx.saved_variables

        kH, kW = ctx.kernel_size
        patchH, patchW = ctx.patch_size
        padH, padW = ctx.padding
        dilationH, dilationW = ctx.dilation
        dilation_patchH, dilation_patchW = ctx.dilation_patch
        dH, dW = ctx.stride

        grad_input1, grad_input2 = correlation.backward(input1, input2, grad_output,
                                                        kH, kW, patchH, patchW,
                                                        padH, padW, dilationH, dilationW,
                                                        dilation_patchH, dilation_patchW,
                                                        dH, dW)
        return grad_input1, grad_input2, None, None, None, None, None, None

In [ ]:
device = "cpu"
batch_size = 1
channel = 1
H = 10
W = 10
dtype = torch.float32

input1 = torch.randint(1, 4, (batch_size, channel, H, W), dtype=dtype, device=device, requires_grad=True)
input2 = torch.randint_like(input1, 1, 4).requires_grad_(True)

In [ ]:
out = spatial_correlation_sample(input1, input2, kernel_size=3, patch_size=1, stride=2, padding=0, dilation=2, dilation_patch=1)

3 3 1 1 0 0 2 2 1 1 2 2


# Lib version

In [ ]:
import torch
import spatial_correlation_sampler
from spatial_correlation_sampler import SpatialCorrelationSampler, spatial_correlation_sample

In [ ]:
import torch
from spatial_correlation_sampler import SpatialCorrelationSampler, spatial_correlation_sample

device = "cpu"
batch_size = 1
channel = 1
H = 10
W = 10
dtype = torch.float32

input1 = torch.randint(1, 4, (batch_size, channel, H, W), dtype=dtype, device=device, requires_grad=True)
input2 = torch.randint_like(input1, 1, 4).requires_grad_(True)

#You can either use the function or the module. Note that the module doesn't contain any parameter tensor.

#function

out = spatial_correlation_sample(input1, input2, kernel_size=3, patch_size=1, stride=2, padding=0, dilation=2, dilation_patch=1)

#module

correlation_sampler = SpatialCorrelationSampler(
    kernel_size=1,
    patch_size=21,
    stride=1,
    padding=0,
    dilation=1,
    dilation_patch=2)
out = correlation_sampler(input1, input2)

In [ ]:
import torch.nn.functional as F
def correlate(input1, input2):
    correlation_sampler = SpatialCorrelationSampler(
                                          kernel_size=1,
                                          patch_size=7,
                                          stride=1,
                                          padding=0,
                                          dilation_patch=2)
    out_corr = correlation_sampler(input1, input2)
    # collate dimensions 1 and 2 in order to be treated as a
    # regular 4D tensor
    b, ph, pw, h, w = out_corr.size()
    out_corr = out_corr.view(b, ph * pw, h, w)/input1.size(1)
    return F.leaky_relu_(out_corr, 0.1)

In [ ]:
correlate(input1, input2).size()

torch.Size([1, 49, 10, 10])

In [ ]:
import torch
import torch.nn as nn

# compile_with_cache Case
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

    def forward(self, input1, input2):
        correlation_sampler = SpatialCorrelationSampler(
            kernel_size=3,
            patch_size=1,
            stride=2,
            padding=0,
            dilation=2,
            dilation_patch=1)
        return correlation_sampler(input1, input2)
        # return spatial_correlation_sample(input1, input2, kernel_size=3, patch_size=1, stride=2, padding=0, dilation=2, dilation_patch=1)
        # return traced_correlation_sampler(input1, input2, kernel_size=3, patch_size=1, stride=2, padding=0, dilation=2, dilation_patch=1)

In [ ]:
net = Net()
trace_model = torch.jit.trace(net, [input1, input2])

RuntimeError: ignored

In [ ]:
torch.jit.save(trace_model, "trace_model.pt")

RuntimeError: ignored

# Reference

stackoverflow
https://stackoverflow.com/questions/26053982/setup-script-exited-with-error-command-x86-64-linux-gnu-gcc-failed-with-exit

https://askubuntu.com/questions/1219870/error-command-x86-64-linux-gnu-gcc-failed-with-exit-status-1-for-lssl-and-lc

solutions:

!apt-get install libssl-dev portaudio19-dev python3.7-dev
